In [1]:
import pandas as pd
import json
from pymongo import MongoClient

In [2]:
fast_food = pd.read_csv("restaurants.csv")
fast_food.head()

,id,dateAdded,dateUpdated,address,categories,primaryCategories,city,country,keys,latitude,longitude,name,postalCode,province,sourceURLs,websites
0,AWrSh_KgsVYjT2BJAzaH,2019-05-19T23:58:05Z,2019-05-19T23:58:05Z,2555 11th Avenue,"Fast Food Restaurants,Hamburgers and Hot Dogs,...",Accommodation & Food Services,Greeley,US,us/co/greeley/255511thavenue/554191587,40.39629,-104.69699,Carl's Jr.,80631,CO,https://www.yellowpages.com/greeley-co/mip/car...,https://www.carlsjr.com/?utm_source=Yextandutm...
1,AWEKlA-LIxWefVJwxG9B,2018-01-18T18:30:23Z,2019-05-19T23:45:05Z,2513 Highway 6 And 50,"Restaurant,Mexican Restaurants,Fast Food Resta...",Accommodation & Food Services,Grand Junction,US,us/co/grandjunction/2513highway6and50/1550891556,39.08135,-108.58689,Del Taco,81505,CO,http://www.citysearch.com/profile/772076870/gr...,http://www.deltaco.com
2,AWrSfAcYsVYjT2BJAzPt,2019-05-19T23:45:04Z,2019-05-19T23:45:04Z,1125 Patterson Road,"Sandwich Shops,Fast Food Restaurants,Restauran...",Accommodation & Food Services,Grand Junction,US,us/co/grandjunction/1125pattersonroad/-2137447852,39.09148,-108.55411,Which Wich,81506,CO,https://www.yellowpages.com/grand-junction-co/...,http://www.whichwich.com
3,AWrSa3NAQTFama1Xpkbz,2019-05-19T23:26:58Z,2019-05-19T23:26:58Z,3455 N Salida Court,"Fast Food Restaurants,Mexican Restaurants,Rest...",Accommodation & Food Services,Aurora,US,us/co/aurora/3455nsalidacourt/1143321601,39.76369,-104.77671,Chipotle Mexican Grill,80011,CO,https://www.yellowpages.com/aurora-co/mip/chip...,http://www.chipotle.com
4,AWrSaVGzZ4Yw-wtdgcaB,2019-05-19T23:24:38Z,2019-05-19T23:24:38Z,5225 E Colfax Avenue,"Fast Food Restaurants,Mexican Restaurants,Rest...",Accommodation & Food Services,Denver,US,us/co/denver/5225ecolfaxavenue/-864103396,39.74044,-104.92636,Taco Bell,80220,CO,https://www.yellowpages.com/denver-co/mip/taco...,https://locations.tacobell.com/co/denver/5225-...


In [3]:
fast_food = fast_food[["address", "latitude", "longitude", "name", "province", "city"]]
fast_food

,address,latitude,longitude,name,province,city
0,2555 11th Avenue,40.396290,-104.696990,Carl's Jr.,CO,Greeley
1,2513 Highway 6 And 50,39.081350,-108.586890,Del Taco,CO,Grand Junction
2,1125 Patterson Road,39.091480,-108.554110,Which Wich,CO,Grand Junction
3,3455 N Salida Court,39.763690,-104.776710,Chipotle Mexican Grill,CO,Aurora
4,5225 E Colfax Avenue,39.740440,-104.926360,Taco Bell,CO,Denver
...,...,...,...,...,...,...
9995,1803 Carolina Ave,33.399574,-86.951050,Church's Chicken Of Birmingham,AL,Bessemer
9996,8511 Cantilever Way Ste 105,35.884500,-78.737541,Luxury Box,NC,Raleigh
9997,2601 Plover Rd,44.453180,-89.543170,McDonald's,WI,Plover
9998,170 E Main St,36.308262,-86.607339,Pizza Hut,TN,Hendersonville


In [4]:
fast_food = fast_food.rename(columns={"province": "state"})
fast_food

,address,latitude,longitude,name,state,city
0,2555 11th Avenue,40.396290,-104.696990,Carl's Jr.,CO,Greeley
1,2513 Highway 6 And 50,39.081350,-108.586890,Del Taco,CO,Grand Junction
2,1125 Patterson Road,39.091480,-108.554110,Which Wich,CO,Grand Junction
3,3455 N Salida Court,39.763690,-104.776710,Chipotle Mexican Grill,CO,Aurora
4,5225 E Colfax Avenue,39.740440,-104.926360,Taco Bell,CO,Denver
...,...,...,...,...,...,...
9995,1803 Carolina Ave,33.399574,-86.951050,Church's Chicken Of Birmingham,AL,Bessemer
9996,8511 Cantilever Way Ste 105,35.884500,-78.737541,Luxury Box,NC,Raleigh
9997,2601 Plover Rd,44.453180,-89.543170,McDonald's,WI,Plover
9998,170 E Main St,36.308262,-86.607339,Pizza Hut,TN,Hendersonville


In [5]:
fast_food.isnull().sum()

address      0
latitude     0
longitude    0
name         0
state        0
city         0
dtype: int64

In [6]:
fast_food.dtypes

address       object
latitude     float64
longitude    float64
name          object
state         object
city          object
dtype: object

In [7]:
# lowercase, get rid of apostrophes, 1st Capital
names_standardized = []

# https://stackoverflow.com/questions/55902042/python-keep-only-alphanumeric-and-space-and-ignore-non-ascii
values = list("abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZ01234567890123456789 ")

for index, row in fast_food.iterrows():
    for character in row["name"]:
        if character not in values:
            row["name"] = row["name"].replace(character, "")
    row["name"] = row["name"].title()
    names_standardized.append(row["name"])

fast_food["name"] = names_standardized
fast_food["name"]

0                            Carls Jr
1                            Del Taco
2                          Which Wich
3              Chipotle Mexican Grill
4                           Taco Bell
                    ...              
9995    Churchs Chicken Of Birmingham
9996                       Luxury Box
9997                        Mcdonalds
9998                        Pizza Hut
9999                      Burger King
Name: name, Length: 10000, dtype: object

In [9]:
fast_food_dict = fast_food.to_dict(orient="records")
fast_food_dict

[{'address': '2555 11th Avenue',
  'latitude': 40.39629,
  'longitude': -104.69699,
  'name': 'Carls Jr',
  'state': 'CO',
  'city': 'Greeley'},
 {'address': '2513 Highway 6 And 50',
  'latitude': 39.08135,
  'longitude': -108.58689,
  'name': 'Del Taco',
  'state': 'CO',
  'city': 'Grand Junction'},
 {'address': '1125 Patterson Road',
  'latitude': 39.09148,
  'longitude': -108.55411000000001,
  'name': 'Which Wich',
  'state': 'CO',
  'city': 'Grand Junction'},
 {'address': '3455 N Salida Court',
  'latitude': 39.763690000000004,
  'longitude': -104.77671,
  'name': 'Chipotle Mexican Grill',
  'state': 'CO',
  'city': 'Aurora'},
 {'address': '5225 E Colfax Avenue',
  'latitude': 39.74044,
  'longitude': -104.92636,
  'name': 'Taco Bell',
  'state': 'CO',
  'city': 'Denver'},
 {'address': '1350 W Colfax Avenue',
  'latitude': 39.739670000000004,
  'longitude': -105.00466999999999,
  'name': 'Mcdonalds',
  'state': 'CO',
  'city': 'Denver'},
 {'address': '200 16th Street',
  'latitude'

In [10]:
# fast_food.to_csv("fast_food.csv", index=False)

In [11]:
myclient = MongoClient("mongodb://localhost:27017/")
myclient.drop_database("fastfood")
  


In [12]:
# database 
db = myclient["fastfood"]
  
# Created or Switched to collection
collection = db["collection"]

In [13]:
collection.insert_many(fast_food_dict)

In [15]:
collection.find()